In [ ]:
import glob
import numpy as np
import random
import tensorflow.compat.v1 as tf
import os
from sklearn.utils import shuffle
batchSize=400
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import fractional_matrix_power
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
k_cluster = 40
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
dnet=tf.keras.applications.densenet.DenseNet121(include_top=False,weights="imagenet",input_tensor=None,input_shape=None,pooling=None)
from tensorflow.keras.models import Model
import sys
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from munkres import Munkres
import cv2
from numpy import save


def load_data():
    train_set = []
    y= []
    #dim=(48,48)
    counter_i = 0
    for filename in glob.iglob('../../../datasets/ORL/orl-dataset/*/*.pgm'):
        img = cv2.imread(filename, -1)
        #img=cv2.resize(img,dim,interpolation=cv2.INTER_LINEAR)  
        train_set.append(img)
        parts=os.path.split(filename)[0]
        parts=parts.split('/')[10]
        parts=parts.split('s')[1]
        y.append(parts)
        print(counter_i)
        counter_i += 1
    y=np.int32(y)
    y=y-1
    train_set, y = shuffle(train_set, y, random_state=42)
    return train_set,y

def extract_features(vtr):
    for i in range(len(vtr)):
        vtr[i]=vtr[i]/np.linalg.norm(vtr[i],2)
    pca.fit(vtr)
    x=pca.transform(vtr)
    return x

def eig_vec(x_train):
    nev_trans=[]
    SortedEigVec=np.zeros((batchSize,batchSize))
    for l in range (0,len(x_train),batchSize):
        xtr=x_train[l:l+batchSize]
        d=(euclidean_distances(xtr,xtr))
        W=np.zeros((len(xtr),len(xtr)))
        for i in range(len(xtr)):
            for j in range (len(xtr)):
                if i==j:
                    W[i,j]=0
                else:
                    W[i,j]=np.exp(-(d[i,j]**2)/(2*0.09))
        degree=np.diag(sum(W))
        L = degree-W
        eig_val,eig_vec=np.linalg.eig(L)
        dec_eigVal=np.flip(np.sort(eig_val))
        indexArr=(np.flip(np.argsort(eig_val)))
        for i in range (len(indexArr)):
            SortedEigVec[:,i]=eig_vec[:,indexArr[i]]
        SortedEigVec=np.array(SortedEigVec)
        nev_trans.append(SortedEigVec)
    return nev_trans,L,eig_val,eig_vec,dec_eigVal,W

def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    for a in range (1):
        if uc1.shape==uc2.shape:
            m = np.ones([l1, l2])
            for i in range(l1):
                it_i = np.nonzero(c1 == uc1[i])[0]
                for j in range(l2):
                    it_j = np.nonzero(c2 == uc2[j])[0]
                    m_ij = np.intersect1d(it_j, it_i)
                    m[i,j] =  -m_ij.size
        else:
            print('assertion handeled')
            return
            #break
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes)
    if cost_matrix is None:
        return (None,None)
    else:
        m = Munkres()
        indexes = m.compute(cost_matrix)
        mapper = { old: new for (old, new) in indexes }
        new_labels = translate_clustering(labels, mapper)
        new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
        return new_labels,accuracy(new_cm)
    
def arch():
    encoding_dim = 390  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
    input_img = Input(shape=(400,))
    encoded_layer_2 = Dense(397, activation='tanh')(input_img)
    encoded_layer_3 = Dense(395, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(393, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(393, activation='tanh')(encoded)
    decoded_layer_3 = Dense(395, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(397, activation='tanh')(decoded_layer_3)
    decoded = Dense(400, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder


def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T      
    
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q

def asl():
    def myloss(y_true, y_pred):
        l1norm= l1norm_func(y_pred,y_true)
        l2norm= l2norm_func(y_pred,y_true)
        orthonormal= orthonormality_func(y_pred)
        structpreserve=structurePreserve_func(y_pred,alpha_trans1[g][:,:])
        return orthonormal+l1norm+l2norm+structpreserve
    return myloss

def l1norm_func(z,nt):
    q1=0.002*(tf.reduce_sum(tf.abs(tf.subtract(z,nt)),axis=1))
    return q1

def l2norm_func(z,nt):
    q2=0.02*(tf.sqrt(tf.reduce_sum(tf.square(z-nt),axis=1)))
    return q2

def structurePreserve_func(z,ta):
    tz=tf.transpose(z)
    tz=tf.reshape(tz,(encoding_dim,batchSize))
    taa=tf.transpose(ta)
    taz=tf.matmul(tz,taa)
    taz=tf.reshape(taz,(encoding_dim,batchSize))
    q4=0.02*(tf.sqrt(tf.reduce_sum(tf.square(tz-taz))))
    q4=tf.transpose(q4)
    return q4


def orthonormality_func(z):
    z=[(z[i,:]/(tf.norm(z[i,:],2)))for i in range (batchSize)]
    transz=tf.transpose(z)
    q3=tf.tensordot(transz,z,axes=1)
    I=tf.eye(encoding_dim)
    q5=0.002*(tf.sqrt(tf.reduce_sum(tf.square(q3-I))))
    return q5



In [ ]:
train_set,y1=load_data()
train_set=np.array(train_set)
y1=np.array(y1)
train_set=np.stack((train_set,)*3, axis=-1)
train_set = train_set.astype('float32') / 255.
vtr=dnet.predict(train_set)
vtr = vtr.reshape((len(vtr), np.prod(vtr.shape[1:])))
x,y=extract_features(vtr,y)
x_train=x
y=y1

x1=x_train[:216,:]
y1=y[:216,]
x2=x_train[296:,:]
y2=y[296:,]
x300=np.concatenate((x1,x2),axis=0)
y300=np.concatenate((y1,y2),axis=0)
idx = np.random.choice(np.arange(len(x300)), 80, replace=False)
x_sample = x300[idx]
y_sample = y300[idx]

xtr=np.concatenate((x_sample,x300),axis=0)
ytr=np.concatenate((y_sample,y300),axis=0)
print(xtr.shape,ytr.shape,np.unique(ytr))

# np.save('orltraindataforSb.npy',xtr)
# np.save('orltrainlabelsforSb.npy',ytr)

x_test=x_train[216:296,:]
y_test=y[216:296,]
# np.save('orltestdata.npy',x_test)
# np.save('orltestlabels.npy',y_test)
# np.unique(y_test)

In [ ]:
nev_trans,L,eig_val,eig_vec,dec_eigVal,W=eig_vec(x_train)
alpha_trans1=np.load('../../../datasets/ORL/Sb_orltrain.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32') 

In [ ]:

encoder,encoding_dim,autoencoder=arch()
autoencoder.compile(tf.keras.optimizers.Adam(lr=1e-3), loss=tf.keras.losses.MeanSquaredError())
autoencoder.fit(x_train,x_train,epochs=100,shuffle=True,batch_size=batchSize)
encoder.save_weights('../../../new_checkpoints/nchk_orl/nE_orltrain.ckpt')

print('training from scratch')
encoder.load_weights('../../../new_checkpoints/nchk_orl/nE_orltrain.ckpt')
g=0
encoder.compile(tf.keras.optimizers.Adadelta(lr=1e-3, rho=0.95, decay=1e-4), loss=asl())

acc=0
for t in range (1,4000,1):
    g=0
    for l in range (0,len(x_train),batchSize):
        encoder.fit(x_train[l:l+batchSize],nev_trans[g][:,batchSize-encoding_dim-1:batchSize-1],epochs=1,shuffle=False,batch_size=batchSize)
        g=g+1
    z_test=encoder.predict(x_train)
    k_means = KMeans(n_clusters=k_cluster, init='k-means++')
    ic= k_means.fit(z_test).cluster_centers_ 
    assignment = assign_func(z_test, ic)
    predictedlabels = np.argmax(assignment, axis=1)
    rectified_label, num=rectify_label(predictedlabels, y)
    if rectified_label is not None:
        print('acc at',t, 'epoch is:',acc)
        if acc<num:
            acc=num
            encoder.save_weights('../../../new_checkpoints/nchk_orl/nsdsc_orltrain.ckpt')
            np.save('../../../new_checkpoints/nchk_orl/nsdsc_icorltrain.npy',ic)
            nmi=sklearn.metrics.normalized_mutual_info_score(y, rectified_label)
            f1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
            prec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')
    else:
        continue

In [ ]:
print(acc,nmi,f1,prec)